In [1]:
# Filename: stock_price_prediction.py

import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error


In [2]:
# Function to preprocess the data
def preprocess_data(stock_data, time_step=60):
    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    stock_data_scaled = scaler.fit_transform(stock_data)

    # Create the dataset with the specified time window
    X, y = [], []
    for i in range(time_step, len(stock_data_scaled)):
        X.append(stock_data_scaled[i - time_step:i, 0])  # The past `time_step` data points
        y.append(stock_data_scaled[i, 0])  # The next day's stock price

    # Convert lists to NumPy arrays
    X, y = np.array(X), np.array(y)

    # Reshape X to be [samples, time steps, features] for LSTM
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Return the processed X, y, and scaler for future inverse transformation
    return X, y, scaler

In [3]:
# Build LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))  # Output layer for stock price prediction
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [4]:

# Train the model and predict
def train_and_predict_for_company(ticker):
    print(f"Fetching data for {ticker}...")

In [ ]:
# Fetch stock data using yfinance
stock_data = yf.download('AAPL', start='2024-01-01', end='2024-09-01')



# Only use 'Close' prices from the stock data
stock_data = stock_data[['Close']]

# Display the first few rows to verify the data
print(stock_data)

In [ ]:
# Fetch stock data using yfinance
stock_data = yf.download('MSFT', start='2024-01-01', end='2024-09-01')
# Only use 'Close' prices from the stock data
stock_data = stock_data[['Close']]

# Display the first few rows to verify the data
print(stock_data)

In [ ]:
# Fetch stock data using yfinance
stock_data = yf.download('GOOGL', start='2024-01-01', end='2024-09-01')
# Only use 'Close' prices from the stock data
stock_data = stock_data[['Close']]

# Display the first few rows to verify the data
print(stock_data)


In [ ]:
# Fetch stock data using yfinance
stock_data = yf.download('NVDA', start='2024-01-01', end='2024-09-01')
# Only use 'Close' prices from the stock data
stock_data = stock_data[['Close']]

# Display the first few rows to verify the data
print(stock_data)


In [9]:
# Split data into training and testing (80% training, 20% testing)
training_data_len = int(len(stock_data) * 0.8)
training_data = stock_data[:training_data_len]
testing_data = stock_data[training_data_len:]


In [10]:
  # Preprocess training data
time_step = 60
X_train, y_train, scaler = preprocess_data(training_data, time_step)


In [ ]:
# Build the LSTM model
model = build_lstm_model((X_train.shape[1], 1))


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32)

In [ ]:
# Preprocess test data
total_data = pd.concat((training_data, testing_data), axis=0)
test_inputs = total_data[len(total_data) - len(testing_data) - time_step:].values
test_inputs_scaled = scaler.transform(test_inputs)

X_test = []
for i in range(time_step, len(test_inputs_scaled)):
        X_test.append(test_inputs_scaled[i-time_step:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Predict the stock prices
predicted_stock_prices = model.predict(X_test)
predicted_stock_prices = scaler.inverse_transform(predicted_stock_prices)

In [15]:
# Get actual stock prices for comparison
actual_stock_prices = testing_data['Close'].values

In [16]:
def train_and_predict_for_company(company):
    # After training and predicting, plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(actual_stock_prices, color='blue', label='Actual Stock Price')
    plt.plot(predicted_stock_prices, color='red', label='Predicted Stock Price')
    plt.title(f'{company} Stock Price Prediction')  # Corrected to 'company'
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()

In [17]:
def plot_stock_predictions(actual_stock_prices, predicted_stock_prices, company):
    """
    Function to plot the actual vs. predicted stock prices.

    Parameters:
    - actual_stock_prices: The actual stock prices from the test data.
    - predicted_stock_prices: The predicted stock prices by the model.
    - company: The ticker symbol of the company (used for the plot title).
    """
    # Create a new figure with a specific size
    plt.figure(figsize=(10, 6))

    # Plot the actual stock prices in blue
    plt.plot(actual_stock_prices, color='blue', label='Actual Stock Price')

    # Plot the predicted stock prices in red
    plt.plot(predicted_stock_prices, color='red', label='Predicted Stock Price')

    # Set the title of the plot
    plt.title(f'{company} Stock Price Prediction')

    # Set the labels for the x and y axes
    plt.xlabel('Time')
    plt.ylabel('Stock Price')

    # Add a legend to differentiate between actual and predicted prices
    plt.legend()

    # Show the plot
    plt.show()



In [18]:
def train_and_predict_for_company(company):
    # After training and predicting
    plot_stock_predictions(actual_stock_prices, predicted_stock_prices, company)

Returned Values:

X_train: The training input data. It's an array where each entry is a sequence of 60 days of stock prices.
y_train: The target output data. It's an array where each entry is the stock price on the day immediately following the 60-day window in X_train.

In [19]:
# Train the model and predict for a given company
def train_and_predict_for_company(company):
    print(f"Fetching data for {company}...")

    # Fetch stock data using yfinance
    stock_data = yf.download(company, start='2024-01-01', end='2024-09-01')
    stock_data = stock_data[['Close']]  # Only use 'Close' prices

    # Split data into training and testing (80% training, 20% testing)
    training_data_len = int(len(stock_data) * 0.8)
    training_data = stock_data[:training_data_len]
    testing_data = stock_data[training_data_len:]

    # Preprocess training data
    time_step = 60
    X_train, y_train, scaler = preprocess_data(training_data, time_step)

    # Build the LSTM model
    model = build_lstm_model((X_train.shape[1], 1))

    # Train the model
    model.fit(X_train, y_train, epochs=20, batch_size=32)

    # Preprocess test data
    total_data = pd.concat((training_data, testing_data), axis=0)
    test_inputs = total_data[len(total_data) - len(testing_data) - time_step:].values
    test_inputs_scaled = scaler.transform(test_inputs)

    X_test = []
    for i in range(time_step, len(test_inputs_scaled)):
        X_test.append(test_inputs_scaled[i - time_step:i, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Predict the stock prices
    predicted_stock_prices = model.predict(X_test)
    predicted_stock_prices = scaler.inverse_transform(predicted_stock_prices)

    # Get actual stock prices for comparison
    actual_stock_prices = testing_data['Close'].values

    # Plot the results
    plot_stock_predictions(actual_stock_prices, predicted_stock_prices, company)


In [ ]:
# Main execution loop for all companies
if __name__ == "__main__":
    companies = ['AAPL', 'MSFT', 'GOOGL', 'NVDA']  # List of companies
    for company in companies:
        print(f"Training and predicting for {company}...")
        train_and_predict_for_company(company)

Goal: The goal of this line is to preprocess your training data so that it can be used to train the Long Short-Term Memory LSTM model. The LSTM will learn from sequences of past stock prices to predict the future stock price.
Outcome: X_train contains the past 60-day windows of stock prices, y_train contains the next day's stock price, and the scaler is returned for later use (when predicting future prices).
The code is designed to predict the future stock price of multiple companies using an LSTM model.
It preprocesses the data by scaling it, builds and trains an LSTM model, makes predictions, and then plots the actual vs predicted stock prices.
The program is modular, so you can easily extend it to work with additional companies or modify the model architecture.

In [21]:
# Set stock symbols for the FOUR companies
companies = ['AAPL', 'MSFT', 'GOOGL', 'NVDA']

In [22]:
# Fetch stock data from Yahoo Finance
def fetch_stock_data(ticker, start_date='2024-01-01', end_date='2024-09-01'):
    stock = yf.download(ticker, start=start_date, end=end_date)
    return stock[['Close']]  # We are only interested in the closing prices


In [23]:
# Preprocess the data for the LSTM model
def preprocess_data(stock_data, time_step=60):
    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    stock_data_scaled = scaler.fit_transform(stock_data)

In [ ]:
# Step 1: Fetch stock data
stock_data = yf.download('AAPL', start='2020-01-01', end='2024-09-01')
stock_data1 = yf.download('MSFT', start='2020-01-01', end='2024-09-01')
stock_data2 = yf.download('GOOGL', start='2020-01-01', end='2024-09-01')
stock_data3 = yf.download('NVDA', start='2020-01-01', end='2024-09-01')




# This downloads stock data and selects the 'Close' column for scaling.
# If stock_data is already defined, skip this step.

stock_data = stock_data[['Close']]  # Only the 'Close' column is used for scaling
stock_data1 = stock_data[['Close']]  # Only the 'Close' column is used for scaling
stock_data2 = stock_data[['Close']]  # Only the 'Close' column is used for scaling
stock_data3 = stock_data[['Close']]  # Only the 'Close' column is used for scaling

# Step 2: Initialize the scaler to scale data between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 4))

# Step 3: Fit the scaler to the data and transform it
stock_data_scaled = scaler.fit_transform(stock_data,stock_data1)

# Step 4: Now stock_data_scaled contains the normalized stock prices
print(stock_data_scaled)

In [25]:

# Assuming stock_data is a pandas DataFrame with the stock's closing prices
stock_data = stock_data[['Close']]  # Only the 'Close' column is used for scaling

# Initialize the scaler to scale data between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler to the data and transform it
stock_data_scaled = scaler.fit_transform(stock_data)

# stock_data_scaled is now a NumPy array of scaled stock prices, normalized between 0 and 1

In [26]:
time_step = 60  # Use the past 60 days of data to predict the next day's price
# Target: next day's price
# Input data: previous 60 days
# Create the dataset with a specified time window
X, y = [], []
for i in range(time_step, len(stock_data_scaled)):
    X.append(stock_data_scaled[i - time_step:i, 0])
    y.append(stock_data_scaled[i, 0])

X, y = np.array(X), np.array(y)

# Reshape X to be [samples, time steps, features] for LSTM
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [27]:
 # Create the dataset with a specified time window
X, y = [], []
time_step = 60  # Define the time step variable

for i in range(time_step, len(stock_data_scaled)):
    # Append the last `time_step` data points for X and the next data point for y
   # X is the sequence of the previous 60 days
    # y is the next day's price


    X.append(stock_data_scaled[i - time_step:i, 0])
    y.append(stock_data_scaled[i, 0])

# Convert lists to NumPy arrays
X, y = np.array(X), np.array(y)

  # Reshape X to be [samples, time steps, features] for LSTM
X = np.reshape(X, (X.shape[0], X.shape[1], 1))


In [28]:
# Build the LSTM model
def build_lstm_model(input_shape):
    model = Sequential()

    # First LSTM layer with dropout regularization
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))


In [29]:
def build_lstm_model(input_shape):
    # Initialize the Sequential model
    model = Sequential()

In [30]:
# Plot stock price predictions vs actuals
def plot_predictions(actual, predicted, company_name):
    plt.figure(figsize=(10, 6))
    plt.plot(actual, color='blue', label='Actual Stock Price')
    plt.plot(predicted, color='red', label='Predicted Stock Price')
    plt.title(f'{company_name} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()


In [ ]:
# Plot the results for Apple
plot_predictions(actual_stock_prices, predicted_stock_prices, 'AAPL')

In [32]:
# Train the model and make predictions for each company
def train_and_predict_for_company(ticker):
    # Fetch stock data
    stock_data = fetch_stock_data(ticker)

In [33]:
 # Split data into training and test sets (80% training, 20% testing)
training_data_len = int(len(stock_data) * 0.8)
training_data = stock_data[:training_data_len]
test_data = stock_data[training_data_len:]

In [34]:

# Split the data into training and test sets
training_data_len = int(len(stock_data) * 0.8)
training_data = stock_data[:training_data_len]

In [35]:

    # Get the actual stock prices (for plotting and comparison)
actual_stock_prices = test_data['Close'][time_step:].values

In [ ]:
# Main execution loop for all companies
if __name__ == "__main__":
    for company in companies:
        print(f"Training and predicting for {company}...")
        train_and_predict_for_company(company)




10 day 50 day moving average

In [37]:
# Function to calculate technical indicators (e.g., moving averages)
def add_technical_indicators(df):
    df.loc[:, '10_day_MA'] = df['Close'].rolling(window=10).mean()  # 10-day moving average
    df.loc[:, '50_day_MA'] = df['Close'].rolling(window=50).mean()  # 50-day moving average
    df.loc[:, 'Volume'] = df['Volume']  # Volume data already exists
    df = df.dropna()  # Drop rows with NaN values resulting from rolling windows
    return df

In [38]:
# Function to preprocess stock data for LSTM with multiple features
def preprocess_data(stock_data, time_step=60):
    # Normalize the data for each feature
    scaler = MinMaxScaler(feature_range=(0, 1))
    stock_data_scaled = scaler.fit_transform(stock_data)

    X, y = [], []
    for i in range(time_step, len(stock_data_scaled)):
        X.append(stock_data_scaled[i - time_step:i])
        y.append(stock_data_scaled[i, 0])  # Only predict the 'Close' price

    X, y = np.array(X), np.array(y)
    return X, y, scaler

In [39]:
# Function to build the LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))  # Output layer for stock price prediction
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [40]:
# Function to plot the actual vs predicted stock prices
def plot_predictions(actual, predicted, company_name):
    plt.figure(figsize=(10, 6))
    plt.plot(actual, color='blue', label='Actual Stock Price')
    plt.plot(predicted, color='red', label='Predicted Stock Price')
    plt.title(f'{company_name} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()


In [41]:
# Function to train the model and predict for a given company
def train_and_predict_for_company(company):
    print(f"Fetching data and training model for {company}...")

    # Fetch stock data using yfinance
    stock_data = yf.download(company, start='2015-01-01', end='2023-01-01')
    stock_data = stock_data[['Close', 'Volume']]  # Include 'Close' and 'Volume'

    # Add technical indicators (e.g., moving averages)
    stock_data = add_technical_indicators(stock_data)

    # Split data into training and testing (80% training, 20% testing)
    training_data_len = int(len(stock_data) * 0.8)
    training_data = stock_data[:training_data_len]
    testing_data = stock_data[training_data_len:]


In [42]:
# Preprocess training data
time_step = 60
X_train, y_train, scaler = preprocess_data(training_data, time_step)



In [ ]:
# Build the LSTM model
input_shape = (X_train.shape[1], X_train.shape[2])  # (time_step, num_features)
model = build_lstm_model(input_shape)

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32)

In [ ]:
# Preprocess test data
total_data = pd.concat((training_data, testing_data), axis=0)
test_inputs = total_data[len(total_data) - len(testing_data) - time_step:].values
test_inputs_scaled = scaler.transform(test_inputs)

X_test = []
for i in range(time_step, len(test_inputs_scaled)):
        X_test.append(test_inputs_scaled[i - time_step:i])
X_test = np.array(X_test)


In [ ]:
# Predict the stock prices
predicted_stock_prices = model.predict(X_test)
predicted_stock_prices = scaler.inverse_transform(predicted_stock_prices)[:, 0]  # Only reverse the scaling for 'Close'


In [47]:
  # Get actual stock prices for comparison
actual_stock_prices = testing_data['Close'].values

In [ ]:
 # Plot the results
plot_predictions(actual_stock_prices, predicted_stock_prices, company)

This code adds technical indicators to stock data:

It computes the 10-day and 50-day moving averages to capture short-term and long-term price trends.
The stock volume is retained as a feature.
The code ensures safe modifications by working on a copy of the DataFrame and removes rows with missing values after the rolling calculations.
The function enhances stock data for use in further analysis or machine learning models, such as predicting future stock prices.

In [49]:
# Function to calculate technical indicators (e.g., moving averages)
def add_technical_indicators(df):
    # Make a copy of the DataFrame to avoid the SettingWithCopyWarning
    df = df.copy()

    # Add 10-day moving average
    df['10_day_MA'] = df['Close'].rolling(window=10).mean()

    # Add 50-day moving average
    df['50_day_MA'] = df['Close'].rolling(window=50).mean()

    # Add volume (no modification needed, just referencing existing data)
    df['Volume'] = df['Volume']

    # Drop rows with NaN values created by rolling windows
    df = df.dropna()

    return df

In [ ]:
# Main execution loop
if __name__ == "__main__":
    companies = ['AAPL', 'MSFT', 'GOOGL']  # List of companies
    for company in companies:
        print(f"Training and predicting for {company}...")
        train_and_predict_for_company(company)

Summary: What the LSTM Model is Doing
This Long Short-Term Memory (LSTM) model is designed to predict stock prices based on historical data (like the past closing prices, volume, and moving averages). It uses deep learning to analyze sequences of stock prices and learn temporal patterns over time.

Key Steps the Model Performs:
Data Collection:






In Summary:
Input: Past stock data (closing prices, volume, etc.).
Output: Predicted future stock prices.
Learning: Temporal patterns in stock prices using LSTM.
Evaluation: Mean Absolute Error (MAE) and Mean Squared Error (MSE) on test data.
Visualization: Actual vs. predicted prices plotted to assess performance.

In [51]:
# Preprocess the test data for X_test and y_test
X_test = []
y_test = []
for i in range(time_step, len(test_inputs_scaled)):
    X_test.append(test_inputs_scaled[i - time_step:i])
    y_test.append(test_inputs_scaled[i, 0])  # Predict the 'Close' price

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Fit the model with training and validation data
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)


model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss and MAE over epochs
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.title('Model Loss and MAE Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss / MAE')
plt.legend()
plt.show()

Fetches recent stock news from Yahoo Finance using the yfinance library, then performs sentiment analysis on the fetched news headlines using a pre-trained Hugging Face model. Here's a breakdown of the main components Positive and Negative sentiment analysis Hugging Face Transformer Library

In [54]:
# Import the Hugging Face pipeline for sentiment analysis
from transformers import pipeline


In [ ]:
# Initialize the sentiment analysis pipeline
sentiment_analysis = pipeline("sentiment-analysis")


In [56]:
# Example texts to analyze sentiment
texts = [
    "The stock market is performing really well today!",
    "I'm very disappointed with the company's financial results.",
    "The product launch was a huge success!",
    "Investors are worried about the economic outlook."
]

In [57]:

# Perform sentiment analysis on the texts
results = sentiment_analysis(texts)


In [ ]:
# Print out the results
for i, text in enumerate(texts):
    print(f"Text: {text}")
    print(f"Sentiment: {results[i]['label']}, Score: {results[i]['score']:.4f}")
    print("----------------------------------------------------")

In [59]:
import yfinance as yf
from transformers import pipeline

In [ ]:

# Initialize the sentiment analysis pipeline from Hugging Face
sentiment_analysis = pipeline("sentiment-analysis")

In [61]:
# Function to fetch recent stock-related news using yfinance
def fetch_stock_news(stock_symbol, num_articles=5):
    stock = yf.Ticker(stock_symbol)
    news = stock.news  # Get stock news

    if not news:
        return []  # Return an empty list if no news is available

    # Extract top `num_articles` news articles
    news_headlines = [article['title'] for article in news[:num_articles]]
    return news_headlines

In [62]:
 # Perform sentiment analysis on stock news headlines
def analyze_sentiment_on_news(stock_symbol, num_articles=5):
    # Fetch stock news
    headlines = fetch_stock_news(stock_symbol, num_articles)

    # Check if headlines are available
    if not headlines:
        print(f"No news found for {stock_symbol}")
        return

    print(f"News headlines for {stock_symbol}:\n")

    # Perform sentiment analysis on each headline
    results = sentiment_analysis(headlines)

    # Print results
    for i, headline in enumerate(headlines):
        sentiment = results[i]['label']
        score = results[i]['score']
        print(f"Headline: {headline}")
        print(f"Sentiment: {sentiment}, Score: {score:.4f}")
        print("----------------------------------------------------")



In [ ]:
# Example: Analyze sentiment on Apple (AAPL) stock news
stock_symbol = "AAPL"
analyze_sentiment_on_news(stock_symbol, num_articles=5)

In [ ]:
 #Example: Analyze sentiment on Google (GOOGL) stock news
stock_symbol = "GOOGL"
analyze_sentiment_on_news(stock_symbol, num_articles=5)

In [ ]:
#Example: Analyze sentiment on Nvidia (NVDA) stock news
stock_symbol = "NVDA"
analyze_sentiment_on_news(stock_symbol, num_articles=5)

In [66]:
import yfinance as yf
from transformers import pipeline
import gradio as gr


In [ ]:

# Initialize the sentiment analysis pipeline from Hugging Face
sentiment_analysis = pipeline("sentiment-analysis")

In [68]:

# Function to fetch recent stock-related news using yfinance # Get stock news  # Return an empty list if no news is available
def fetch_stock_news(stock_symbol, num_articles=5):
    stock = yf.Ticker(stock_symbol)
    news = stock.news  
    
    if not news:
        return [] 
    
    # Extract top `num_articles` news articles
    news_headlines = [article['title'] for article in news[:num_articles]]
    return news_headlines


In [69]:
# Gradio interface function
def gradio_interface(stock_symbol, num_articles):
    return analyze_sentiment_on_news(stock_symbol, num_articles)

# Create Gradio interface with the updated components # Function to run # Output will be text
iface = gr.Interface(
    fn=gradio_interface,  
    inputs=[gr.Textbox(label="Stock Symbol", placeholder="AAPL"), 
            gr.Slider(1, 10, value=5, label="Number of News Articles")],
    outputs=gr.Textbox(),  
    title="Stock News Analysis",
    description="Enter a stock symbol (e.g., AAPL) and the number of news articles to analyze."
)


In [ ]:

# Launch the Gradio interface #public link  = iface.launch(share=True)
iface.launch(share=True)

In [1]:
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEndpoint
from langchain_huggingface import HuggingFacePipeline

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="ByteWave/prompt-generator")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
